In [514]:
# the following code allows you to import function from
# jupyter notebook in other directory

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [515]:
import functools
%run ../gw/gw_wrapped_compressed.ipynb
%run ../compressed_class_list/compressed_class_list.ipynb

IndexError: list index out of range

In [516]:
class ChiangOGWInvariant():
    def __init__(self, use_recursion_a=True, ogw_1_0_g2=var('x'), debug=False):
        self.use_recursion_a = use_recursion_a
        self.ogw_1_0_g2 = ogw_1_0_g2
        #self.ogw_1_0_g2 = 27/16
        self.debug = debug

    @functools.lru_cache(maxsize=None)
    def chiang_ogw_invariant(self, n, b, k, A, layer=0):
        #print(f"b:{b}, k:{k}, A:{A}")
        if isinstance(A, list):
            A = ClassList.compress(A, n)

        l = len(A)      
        if A.invalid_class_list:
            return 0    

        # wall-crossing theorem (Elad)
        if l > 0 and A[0] == -1:
            return -self.chiang_ogw_invariant(n, b, k+1, A[:-1]) 

        #degree axiom (Elad)
        if 2*sum(A) != (n - 3 + 2*b + k + 2*l - k*n):  
            # The maslov index is twice the chern class n+1 for holomorphic spheres. 
            # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
            # p 595 mcduff 
            #ogw_d[str((n, b, k, A)) + ", degree axiom"] = 0
            if layer == 1:
                print("degree axiom fails")
            return 0    

        # other
        #if k == 0 and is_even(b):
        #    return 0    

        # zero axiom
        if b == 0:
            if k == 1 and l == 1 and A[0] == 0:
                return -1
            else:
                return 0    

        # unit axiom (May)
        # handle all beta = 0 cases
        if l > 0 and A[0] == 0:
            if b == 0 and k == 1 and l == 1:
                return -1
            else:
                return 0    

        # divisor axiom 
        if l > 0 and A[0] == 1:
            #A.pop(0)
            return (b/4)*self.chiang_ogw_invariant(n, b, k, A[1:])   

        # base cases
        if b == 1 and k == 1 and A == []:
            return 3    

        if b == 1 and k == 0 and A == [2]:
            return self.ogw_1_0_g2
        
        if b == 2 and k == 2 and A == []:
            return 2
        
        # TODO: hand check
        if b == 2 and k == 0 and A == [2, 2]:
            return (-1 + self.chiang_ogw_invariant(n, 2, 1, ClassList.compress([2], n))^2)\
                    /(self.chiang_ogw_invariant(n, 2, 2, ClassList.compress([], n)))
        
        # TODO : hand check
        if b == 2 and k == 1 and A == [2]:
            return -self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))\
                    *self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))
        
        if b == 2 and k == 0 and A == [3]:
            return 2*(self.chiang_ogw_invariant(n, 2, 0, ClassList.compress([2, 2], n)) \
                    + self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))*\
                        self.chiang_ogw_invariant(n, 1, 0, ClassList.compress([1, 2], n)))
             

        # recursion (a)
        if l >= 2 and not (k >= 1 and not self.use_recursion_a):
            cor = self.chiang_ogw_invariant(n, b, k, [A[0]-1, A[1]+1] + A[2:])
            col = 0
            ool = 0
            oor = 0
            A_0 = A[0]
            A_1 = A[1]  

            for t in A[2:].bipart():
                I1 = t[0]
                I2 = t[1]
                bipart_coeff = ClassList.bipart_coeff(A[2:], I1)    

                for i in range(0, n+1):
            
                    L1_cor = [1, A_1, i] + I1
                    L2_cor = [n-i, A_0-1] + I2 
                    for b1 in range(1, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue
                        
                        cor += bipart_coeff \
                            * (-1)^(b_*(n+1)/2) *  gw_invariant(n, b_, L1_cor) \
                            *  self.chiang_ogw_invariant(n, b1, k, L2_cor)   

                for i in range(0, n+1):
                
                    L1_col = [1, A[0] - 1, i] + I1 
                    L2_col = [n-i, A[1]] + I2 
                    for b1 in range(1, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue
                        
                        col += bipart_coeff \
                            * (-1)^((n+1)*b_/2)* gw_invariant(n, b_, L1_col) \
                            * self.chiang_ogw_invariant(n, b1, k, L2_col)    

                L1_ool = [1, A[0]-1] + I1
                L2_ool = [A[1]] + I2
                for b1 in range(1, b):
                    b2 = b - b1 

                    for k1 in range(0, k+1):
                        k2 = k - k1
                        ool += bipart_coeff \
                            * binomial(k, k1)*self.chiang_ogw_invariant(n, b1, k1, L1_ool) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, L2_ool)   

                L1_oor = [1, A[1]] + I1
                L2_oor = [A[0] - 1] + I2    
                for b1 in range(1, b):
                    b2 = b - b1 

                    for k1 in range(0, k+1):
                        k2 = k - k1 

                        oor += bipart_coeff \
                            * binomial(k, k1) * self.chiang_ogw_invariant(n, b1, k1, L1_oor) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, L2_oor)   

            result = cor - col - oor + ool
            if self.debug:
                print(f"recursion a: {result}")
            return result
        
        # recursion (c)
        if l >= 1 and k >= 1:
            #print(f"used c, A:{A}")
            #print(f"(c): b:{b}, k:{k}")
            co = 0
            ool = 0
            oor = 0
            A_0 = A[0]  

            for t in A[1:].bipart():
                I1 = t[0]
                I2 = t[1]   

                bipart_coeff = ClassList.bipart_coeff(A[1:], I1)    

                for i in range(0, n+1):
                    for b1 in range(0, b): # make sure that b1 is never b
                        b_ = 0 # \hat{\beta}
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue

                        co += bipart_coeff \
                            * gw_invariant(n, b_, [A_0 - 1, 1, i] + I1) \
                            * self.chiang_ogw_invariant(n, b1, k, [n - i] + I2)   

                for b1 in range(1, b+1): # zero when b1 = 0
                    b2 = b - b1 

                    for k1 in range(0, (k-1)+1):
                        k2 = k - 1 - k1
                        ool += bipart_coeff \
                            * binomial(k-1, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1, [A_0 - 1, 1] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 2, I2)   

                for b1 in range(1, b+1): # exclude b1 = 0, which is zero, but would still cause infinite recursion
                    b2 = b - b1

                    for k1 in range(0, (k-1)+1):
                        k2 = k - 1 - k1
                        oor += bipart_coeff \
                            * binomial(k-1, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1 + 1, [A_0 - 1] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, [1] + I2) 
                        
            #print(f"recursion c,  b:{b}, k:{k}, A:{A}, co:{co}, ool:{ool}, oor:{oor}")

            result =  -co + ool - oor
            if self.debug:
                print(f"recursion c : {result}")
            return result   

        
        # recursion (d)
        if k >= 2:
            co = 0
            ool = 0
            oor = 0
            
            for t in A.bipart():
                I1 = t[0]
                I2 = t[1]   

                bipart_coeff = ClassList.bipart_coeff(A, I1)    

                for i in range(0, n+1):
                    for b1 in range(0, (b+2)+1): # make sure that b1 is never b
                        b_ = 0
                        if ((b + 2) - b1) % 4 == 0:
                            b_ = ((b + 2) - b1) / 4
                            if b_ == 0:
                                continue
                        else:
                            continue
                        co += bipart_coeff \
                            * gw_invariant(n, b_, [2, 2, i] + I1) \
                            * self.chiang_ogw_invariant(n, b1, k-1, [n - i] + I2)
                        
                for b1 in range(2, b+1):
                    b2 = (b + 2) - b1
                    for k1 in range(0, (k-2)+1):
                        k2 = (k - 2) - k1
                        
                        if b1 == b and k1 == k - 2:
                            continue
                            
                        ool += bipart_coeff \
                            * binomial(k-2, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1+2, I1) \
                            * self.chiang_ogw_invariant(n, b2, k2, [2, 2] + I2)
                
                for b1 in range(2, b+1): # zero when b1 = 0
                    b2 = (b + 2) - b1 

                    for k1 in range(0, (k-2)+1):
                        k2 = (k - 2) - k1
                        oor += bipart_coeff \
                            * binomial(k-2, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1+1, [2] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2+1, [2] + I2)
                    
            
            result = (co - ool + oor)/(self.chiang_ogw_invariant(n, 2, 0, ClassList.compress([2, 2], n)))       
            if self.debug:
                print(f"recursion d: {result}")
            return result   

        print(f"unexpected situation is , b: {b}, k: {k}, A:{A.c_list}, l: {l}")
        return 0

In [517]:
chiang_ogw_c = ChiangOGWInvariant(use_recursion_a=False)

chiang_c = chiang_ogw_c.chiang_ogw_invariant(3, 4, 1, ClassList.compress([2, 2, 2], 3))


In [518]:
# get value that satisfies the degree axiom 
def get_l1(n, b, k, l2, A1, A2):
    m = n - 3 + 2*b + k + 2*l2 - k*n - 2*l2 * A2 # degree axiom
    if (m % (2*(A1 - 1))) == 0:
        return int(m / (2*(A1 - 1)))
    else:
        -1


In [519]:

chiang_ogw = ChiangOGWInvariant(debug=False)
w = chiang_ogw.chiang_ogw_invariant(3, 2, 0, ClassList.compress([2, 2], 3))
print(f"w:{w}")
#b == 2 and k == 0 and A == [2, 2]

w:-175/512


In [520]:
chiang_ogw = ChiangOGWInvariant(debug=False)
# k = 0, no boundary points
for b in range(0, 10):
    for l2 in [1, 3, 5]:
        A1 = 2
        A2 = 3
        l1 = get_l1(3, b, 0, l2, A1, A2)
        if l1 < 0 :
            continue
        A = [A1]*l1 + [A2]*l2
        chiang = chiang_ogw.chiang_ogw_invariant(3, b, 0, ClassList.compress(A, 3))
        print(f"b: {b}, A: {A}, chiang ogw: {chiang}")

b: 2, A: [3], chiang ogw: 3/8*x - 175/256
b: 3, A: [2, 3], chiang ogw: -9/64*x + 525/2048
b: 4, A: [2, 2, 3], chiang ogw: 1/1024*(32*x + 27)*x + 135/1024*x - 7875/32768
b: 5, A: [2, 2, 2, 3], chiang ogw: -5/524288*(96*x - 175)*(32*x + 27) + 1/65536*(96*x - 175)*(8*x + 27) + 3/4096*(96*x - 47)*x - 9/2048*(32*x + 27)*x + 4197/16384*x + 203175/524288
b: 6, A: [2, 2, 2, 2, 3], chiang ogw: 3/524288*(96*x - 47)*(96*x - 175) - 3/1048576*(96*x - 67)*(96*x - 175) + 27/8388608*(192*x - 875)*(32*x + 27) + 165/8388608*(96*x - 175)*(32*x + 27) - 57/2097152*(96*x - 175)*(8*x + 27) - 3/45875200*(32*(64*(16*x - 27)*x + 4725)*x + 1461600*x - 1051575)*x + 3/5734400*(8*(64*(16*x - 27)*x + 4725)*x + 50400*x - 58275)*x - 45/65536*(96*x - 47)*x + 81/8192*(32*x + 27)*x - 127203/262144*x - 5751025/8388608
b: 6, A: [3, 3, 3], chiang ogw: 1/131072*(96*x - 47)*(96*x - 175) - 9/524288*(96*x - 175)*(32*x + 27) + 1/716800*(8*(64*(16*x - 27)*x + 4725)*x + 50400*x - 58275)*x + 9/4096*(32*x + 27)*x - 525/4096*x - 1417

In [521]:
chiang_ogw_c = ChiangOGWInvariant(use_recursion_a=False)

chiang_c = chiang_ogw_c.chiang_ogw_invariant(3, 4, 1, ClassList.compress([2, 2, 2], 3))



In [522]:
x = var('x')
chiang_ogw_a = ChiangOGWInvariant(ogw_1_0_g2=x)
chiang_ogw_c = ChiangOGWInvariant(use_recursion_a=False, ogw_1_0_g2=x)
# experiment k > 0
for b in range(1, 10):
    for k in range(0, 7):
        for l2 in range(0, 7):
            A1 = 2
            A2 = 3
        
            l1 = get_l1(3, b, k, l2, A1, A2)
            # l1 = 1
            if l1 < 0 :
                continue
            A = [A1]*l1 + [A2]*l2
            chiang_a = chiang_ogw_a.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))
            chiang_c = chiang_ogw_c.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))
            
            if chiang_a == chiang_c:
                print(f"b:{b}, k:{k}, A:{A} polynomial agrees")
                continue
            else:
                s = solve([chiang_a == chiang_c], x)
                print(f"b:{b}, k:{k}, A:{A}\nsolution: x = {s}\nchiang_a: {chiang_a}\nchiang_c: {chiang_c} \n")
                
print("Tests Finish!")

b:1, k:0, A:[2] polynomial agrees
b:1, k:1, A:[] polynomial agrees
b:2, k:0, A:[2, 2] polynomial agrees
b:2, k:0, A:[3] polynomial agrees
b:2, k:1, A:[2] polynomial agrees
b:2, k:2, A:[] polynomial agrees
b:3, k:0, A:[2, 2, 2] polynomial agrees
b:3, k:0, A:[2, 3] polynomial agrees
b:3, k:1, A:[2, 2] polynomial agrees
b:3, k:1, A:[3] polynomial agrees
b:3, k:2, A:[2] polynomial agrees
b:3, k:3, A:[] polynomial agrees
b:4, k:0, A:[2, 2, 2, 2] polynomial agrees
b:4, k:0, A:[2, 2, 3] polynomial agrees
b:4, k:0, A:[3, 3] polynomial agrees
b:4, k:1, A:[2, 2, 2] polynomial agrees
b:4, k:1, A:[2, 3] polynomial agrees
b:4, k:2, A:[2, 2] polynomial agrees
b:4, k:2, A:[3] polynomial agrees
b:4, k:3, A:[2] polynomial agrees
b:4, k:4, A:[] polynomial agrees
b:5, k:0, A:[2, 2, 2, 2, 2] polynomial agrees
b:5, k:0, A:[2, 2, 2, 3] polynomial agrees
b:5, k:0, A:[2, 3, 3] polynomial agrees
b:5, k:1, A:[2, 2, 2, 2] polynomial agrees
b:5, k:1, A:[2, 2, 3] polynomial agrees
b:5, k:1, A:[3, 3] polynomial agr

In [523]:
x, y = var('x, y')
a = solve([3/35840*(96*x - 175)*(16*x - 27) - 3/2800*(96*x - 337)*x + 9/1400*(16*x - 27)*x - 27/32*x - 3537/2048
== 1/1400*(96*x - 337)*x + 3/800*(16*x - 27)*x + 1/64*x - 5427/2048
      ],
      x)

c = solve([ -3/8*x + 175/512 ==0
      ],
      x)

print(f"a:{a}, c:{c} ")
#-1/4*x + 27/256, chiang_c: 525/4096



a:[
x == (27/16)
], c:[
x == (175/192)
] 


In [524]:
x, y = var('x, y')
solve([3/44800*(96*x - 175)*(16*x - 27) + 3/700*(16*x - 27)*x - 1/2*x - 189/256
== -1/22400*(96*x - 175)*(16*x - 27) + 3/200*(16*x - 27)*x - 3/16*x - 81/64
      ],
      x)



[x == (27/16)]

In [525]:
x, y = var('x, y')
solve([ 3/44800*(96*x - 175)*(16*x - 27) + 3/700*(16*x - 27)*x - 1/2*x - 189/256
== -1/22400*(96*x - 175)*(16*x - 27) + 3/200*(16*x - 27)*x - 3/16*x - 81/64 ],
      x)
#1/4096*(64*x - 27)*x + 351/2048*x - 61425/131072, chiang_c: 10707/65536*x - 61425/131072

[x == (27/16)]

In [526]:
x, y = var('x, y')
solve([ 9/22937600*(64*(16*x - 27)*x - 4725)*(96*x - 175) - 3/45875200*(64*(16*x - 27)*x - 29925)*(96*x - 175) - 3/2867200*(10*(96*x - 175)*(16*x - 27) - 128*(96*x - 337)*x + 768*(16*x - 27)*x - 100800*x - 206325)*x - 3/358400*(3*(96*x - 175)*(16*x - 27) + 192*(16*x - 27)*x - 22400*x - 33075)*x + 141/1433600*(64*(16*x - 27)*x - 4725)*x - 3/716800*(64*(16*x - 27)*x - 29925)*x - 23/2048*(32*x + 27)*x - 15/4096*(16*x - 27)*x + 1/1024*(8*x + 27)*x - 19197/8192*x + 1021041/524288
== -27/32768*(8*x + 27)^2 + 1/1433600*(256*(96*x - 337)*x + 1344*(16*x - 27)*x + 5600*x - 949725)*x + 9/573440*(64*(16*x - 27)*x - 29925)*x + 1/512*(32*x + 27)*x + 3/2048*(16*x - 27)*x - 1/512*(8*x + 27)*x - 8613/8192*x + 1326519/524288],
      x)


[x == (27/16), x == 0]

In [527]:
x, y = var('x, y')
solve([ -3/8*x + 175/512 ==0
      ],
      x)



[x == (175/192)]

In [528]:
b:4, k:1, A:[2, 3],
            chiang_a: -3/16*x + 175/512,
                chiang_c: 81/1024*x - 4725/32768

SyntaxError: invalid syntax (<ipython-input-528-5e7567b4f044>, line 1)